In [11]:
# Require Pandas version >= 2.2.2
import pandas as pd
assert pd.__version__ >= '2.2.2', 'Please update Pandas to version 2.2.2 or newer'
import pandas as pd, os, pyreadr
import warnings
import time
import numpy as  np
import pandas as pd
import pandas as pd
import visualization as viz
import data_loading as dl
import risk_pricing as rp
import statsmodels.api as sm
import statsmodels.formula.api as smf
import importlib
importlib.reload(rp)
importlib.reload(viz)
importlib.reload(dl)
warnings.filterwarnings("ignore")
source_folder_path = "/Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/src/"
modelname = "embeddings_km10_ipcs" # "dicfullmc10thr10defnob40noa0_8_hdp" # Don't forget to clean again when modelname changes
start_time = time.time()
# print the current working directory (for reference)
print("Current Working Directory:", os.getcwd())
os.chdir(source_folder_path)
print("New Working Directory:", os.getcwd())

Current Working Directory: /Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/src
New Working Directory: /Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/src


In [12]:
# # Creating additional models
# import matplotlib.pyplot as plt
# embeddings = Embeddings()
# EmbeddingsHKRModel.plot_cosine_similarity(["intellectual property", "patent", "knowledge capital", "research and development", "key talent"])
# Save last plot:
# plt.savefig("cosine_similarity.png")
# base_path = "/Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/text/"
# dir_path = os.path.join(base_path)
# os.makedirs(dir_path, exist_ok=True)
# figfolder = os.path.join(dir_path, "")
# # Save to the folder:
# plt.savefig(figfolder + "cosine_similarity_terms.png")
# EmbeddingsHKRModel().from_topic_similarity(embeddings, "emb_knocap", "knowledge capital", to_csv = True)
# EmbeddingsHKRModel().from_topic_similarity(embeddings, "emb_patents", "patents", to_csv = True)
# EmbeddingsHKRModel().from_topic_similarity(embeddings, "emb_intcap", "intangible capital", to_csv = True)

In [13]:
# Loading the data from load_dataframes:


In [14]:
import risk_pricing as rp
import importlib
import embedding_tools as et
from embedding_tools import Embeddings
from embedding_tools import EmbeddingsHKRModel
importlib.reload(rp)
importlib.reload(viz)
quantiles = 10
# embeddings = Embeddings()
# EmbeddingsHKRModel().from_topic_similarity(embeddings, "emb_kt", "key talent", to_csv = True)
for quantiles in [10]:
    for modelname in ["embeddings_km10_ipcs"]:
        for pfname in ["pfkki3me3mb"]:
            amazon_nov01_short, cequity_mapper, ff3fw, ff5fw, ff3fm, ff5fm, topic_map_unlabeled, comparison_measures, stoxmo_orig, stoxda_orig, stoxwe_orig = \
                dl.load_dataframes(modelname, start_time, clean_again = False)
            cuts = np.linspace(0, 1, quantiles+1).tolist()
            topic_map = viz.label_topic_map(topic_map_unlabeled, modelname, cuts = cuts)
            print("Running Fama-MacBeth regressions: 5 factors")
            fmb_5, indexed_df, eret_we, stoxwe_add = rp.famaMacBethFull(stoxwe_orig, cequity_mapper, topic_map, ff5fw, pfname, kki_cuts = cuts, window_size = 52*2, add_innerkk_pf = True)
            fmb_3, _, _, _ = rp.famaMacBethFull(stoxwe_orig, cequity_mapper, topic_map, ff3fw, pfname, kki_cuts = cuts, window_size = 52*2, add_innerkk_pf = True)
            fmb_1, indexed_df = rp.famaMacBeth(eret_we, pfname, formula = "eretw ~ 1 + MktRF + HKR", window_size=52*2)
            print("Running Fama-MacBeth regressions: 1 factor")
            #import visualization as viz
            base_path = "/Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/text/"
            dir_path = os.path.join(base_path, f"{modelname}_{quantiles}tiles_{pfname}_HKR_SB")
            os.makedirs(dir_path, exist_ok=True)
            figfolder = os.path.join(dir_path, "")
            generate_plots = True
            ##
            viz.explore_topic_map(topic_map, figfolder, generate_plots)
            viz.explore_fmb([fmb_1, fmb_3, fmb_5], figfolder)
            viz.explore_eret_we(eret_we, figfolder)
            viz.tex_compare_kk_measures(comparison_measures, figfolder)
            viz.plot_returns(stoxwe_add, figfolder)
            #viz.amazon_graph(amazon_nov01_short, figfolder)
            #viz.filecounter(figfolder)
            viz.explore_stoxda(stoxda_orig, cequity_mapper, topic_map, figfolder)

Tempo de execução depois de carregar os dados: 12.31749415397644
Running Fama-MacBeth regressions: 5 factors
Finished processing stoxwe
New kernel
Finished Fama-MacBeth
Finished processing stoxwe
New kernel
Finished Fama-MacBeth
New kernel
Running Fama-MacBeth regressions: 1 factor
Executing fig_mean_tiy...
Executing tex_average_topic_loadings_by_high_tech...
Executing tex_sample_topic_loadings...
Executing df_skill_correlations...
Executing df_patent_correlations...
Executing fig_share_dominant_kk_by_ind...
Executing fig_heatmap_topicvskkpt...
Executing fig_heatmap_topicvsikpt...
Executing fig_histogram_kk...
Executing fig_histogram_kk_by_ind12...
Finished!
Executing explore_fmb...
Special case for FamaMacBethResults
Special case for FamaMacBethResults
Special case for FamaMacBethResults
Executing fig_h1b_vs_smb_kkhml...
Executing tex_HKR_vs_mktrf...
Now using pseudo-monthly returns
Finished using pseudo-monthly returns!
Executing tex_summary_statistics...
Executing tex_compare_kk_mea

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [15]:
eret_we_agg = (eret_we
        .groupby('yw')
        .mean())
eret_we_agg = eret_we_agg.reset_index()
eret_we_agg.yw = eret_we_agg.yw.astype(int)
def convert_yw_to_date(yw):
    year = yw // 100
    week = yw % 100
    # Create a date corresponding to the first day of the year
    date = pd.to_datetime(year.astype(str) + '0101', format='%Y%m%d')
    # Add the number of weeks to get the first day of the week
    return date + pd.to_timedelta((week - 1) * 7, unit='D')

# Apply the function to the yw column
eret_we_agg['date'] = convert_yw_to_date(eret_we_agg['yw'])
# Keep only columns whose names match one of the set: date, eretw, Mkt.RF, SMB, HML, RMW, CMA, RF, HKR, HKR_NSB
columns_to_keep = {'date', 'eretw', 'Mkt.RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', 'HKR', 'HKR_NSB'}
# Drop columns index, yw, and any other column whose name is not in columns_to_keep
eret_we_agg = eret_we_agg[eret_we_agg.columns.intersection(columns_to_keep)]

columns_to_aggregate = ['eretw', 'Mkt.RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', 'HKR', 'HKR_NSB']

# Define a function to aggregate groups of 4 rows
def aggregate_groups(df):
    # Create a grouping variable
    df['group'] = df.index // 4
    
    # Define aggregation rules
    agg_rules = {'date': ('date', 'last')}
    
    for col in columns_to_aggregate:
        if col in df.columns:
            agg_rules[col] = (col, 'sum')
    
    # Aggregate the groups
    pseudo_monthly = df.groupby('group').agg(**agg_rules).reset_index(drop=True)
    
    return pseudo_monthly

# Apply the aggregation function
pseudo_monthly = aggregate_groups(eret_we_agg)


In [16]:
from statsmodels.iolib.summary2 import summary_col
from risk_pricing import ret_nwks_ahead


eret_we_agg = eret_we_agg[eret_we_agg.yw > 200900]
eret_we_agg = eret_we_agg[eret_we_agg.yw < 202012]
eret_we_agg = ret_nwks_ahead(eret_we_agg, 4)
eret_we_agg = ret_nwks_ahead(eret_we_agg, 12)
eret_we_agg = ret_nwks_ahead(eret_we_agg, 52)
eret_we_agg = ret_nwks_ahead(eret_we_agg, 104)
eret_we_agg = ret_nwks_ahead(eret_we_agg, 156)
#eret_we_agg = ret_nwks_ahead(eret_we_agg, 52*4)
#eret_we_agg = ret_nwks_ahead(eret_we_agg, 52*5)

eret_we_agg.rename(columns = {'Mkt.RF_4w': 'MktRF_4w', 
                            'Mkt.RF_12w': 'MktRF_12w', 
                            'Mkt.RF_52w': 'MktRF_52w', 
                            'Mkt.RF_104w': 'MktRF_104w', 
                            'Mkt.RF_156w': 'MktRF_156w'#, 'Mkt.RF_208w': 'MktRF_208w', 'Mkt.RF_260w': 'MktRF_260w', 
                            }, inplace=True)

summary = (summary_col([smf.ols(formula="MktRF_4w ~ HKR", data=eret_we_agg).fit(cov_type='HAC', cov_kwds={'maxlags': 4}),  # cov_type='HAC', cov_kwds={'maxlags': 4}: Should be added?
                        smf.ols(formula="MktRF_12w ~ HKR", data=eret_we_agg).fit(cov_type='HAC', cov_kwds={'maxlags': 12}),#fit(cov_type='HAC', cov_kwds={'maxlags': 12}), 
                        smf.ols(formula="MktRF_52w ~ HKR", data=eret_we_agg).fit(cov_type='HAC', cov_kwds={'maxlags': 52}),#fit(cov_type='HAC', cov_kwds={'maxlags': 52}), 
                        smf.ols(formula="MktRF_104w ~ HKR", data=eret_we_agg).fit(cov_type='HAC', cov_kwds={'maxlags': 104}),#fit(cov_type='HAC', cov_kwds={'maxlags': 104}), 
                        smf.ols(formula="MktRF_156w ~ HKR", data=eret_we_agg).fit(cov_type='HAC', cov_kwds={'maxlags': 156}),#fit(cov_type='HAC', cov_kwds={'maxlags': 156})#, sm.formula.ols(formula="MktRF_208w ~ HKR", data=eret_we_agg).fit(), sm.formula.ols(formula="MktRF_260w ~ HKR", data=eret_we_agg)
                        ],  # List of regression result objects
                    model_names=['4wa', '12wa', '52wa', '104wa', '156wa'],#, '208wa', '260wa'],  # Names for each model
                    stars=True,  # Include significance stars
                    float_format='%0.4f',  # Format for displaying coefficients
                    regressor_order=['HKR', 'SMB', 'HML'],  # Order of variables in the table
                    drop_omitted=True))  # Drop omitted variables
    # fig_h1b_vs_smb_kkhml(eret_we_agg, figfolder)
    # tex_HKR_vs_mktrf(eret_we_agg, figfolder)
    # tex_summary_statistics(eret_we_agg, figfolder)
summary

ImportError: cannot import name 'ret_nwks_ahead' from 'risk_pricing' (/Users/pedrovallocci/Documents/PhD (local)/Research/Github/KnowledgeKRisk_10Ks/src/risk_pricing/__init__.py)

In [ ]:
summary

,4wa,12wa,52wa,104wa,156wa
SMB,0.0217,0.0940,-0.0574,-0.2265,-0.3702
,(0.1650),(0.2457),(0.2854),(0.3161),(0.2791)
HML,0.1575,-0.1136,0.2038,-0.0826,-0.4292
,(0.1670),(0.2261),(0.3223),(0.3273),(0.2990)
R-squared,0.0027,0.0016,0.0008,0.0006,0.0062
R-squared Adj.,-0.0025,-0.0037,-0.0048,-0.0057,-0.0009
